In [566]:
import numpy as np
import math as mth

I created this notebook in an effort to quantify the baseline risks we face going forward as we learn to "live with covid" in order to help convince people in my social circle to redouble their efforts to avoid getting covid now that the narrative has shifted.  I've tried to use conservative estimates in an effort to model a realistic 'best-case' scenario (I don't want to risk being dismissed as alarmist).  It consists of two sections, one to model cumulative mortality risks and another to model cumulative morbidity risks.

# I. Mortality Risks

This is a simple calculator that you can use to estimate your cumulative covid19 mortality risk over multiple infections spanning several years, based on current [CDC mortality estimates](https://www.cdc.gov/coronavirus/2019-ncov/hcp/planning-scenarios.html).  

```**NB:** The CDC IFR estimates that this model uses are based predominantly on data from initial infections and may therefore not represent actual mortality risk for repeat infections.  In other words, as the pandemic progresses, it may be the case that, at some point, each infection predisposes an individual to worse outcomes from subsequent infections, and if this is the case, then this calculator may significantly understate risk (that is, the risk of mortality for your sixth infection may be greater than your initial risk, in which case, your actual cumulative risk for multiple infections may be much higher than that modelled here).```

In the cell labeled **```2 - PARAMETERS```**, below, enter your current ```age```, the number of ```years``` and the number of infections per year (```infections_per_year```) to model.  For example, to model the mortality risk for a 45 year old that expects to be reinfected an average of once every 18 months over the next ten years you'd enter  
```python
    age = 45
    years = 10
    infections_per_year = 2/3
```
Using the [CDC's estimates of Infection Fatality Rate](https://www.cdc.gov/coronavirus/2019-ncov/hcp/planning-scenarios.html), this function will attempt to determine your updated mortality risk per infection and return an estimate of your cumulative mortality risk for the given time period.  Since this calculator is based on CDC estimates of IFR, which is based on historical data, it doesn't take into account variations in mortality per infection by new variants, nor does it take into account as-yet undiscovered/undeployed pharmaceutical interventions.  

Moreover, as alluded to above, since infection sometimes results in chronic conditions specifically associated with worse outcomes and likely results in some degree of immune "aging", neither of which are modelled here, **this model is, barring new pharmaceutical interventions, likely to underestimate risk to some degree, perhaps quite significantly**, and the results calculated are probably best thought of as a **"best case scenario"** for the parameters modelled.

This calculator does, however, take into account the protective effect of vaccination and/or previous infection - meaning it assumes that your risk of dying from an infection at this point in the pandemic is about 40% lower than it would have been two years ago (when most people were completely immuno-naïve).  This is based on the protective effect of immunity observed thus far in the pandemic, and may once again lead to an underestimate of risk (if, for example, this effect doesn't endure, if new variants completely evade immunity, or if ADE becomes an issue).

Scroll down to the **```EXPLANATION```** section below if you'd like to read about how these estimates are calculated. Otherwise, follow the instructions in cells ```1```-```3``` below.

**BUT FIRST** a note on ```infections_per_year```. Prior to omicron, it had been estimated that, owing to waning immunity and viral evolution, recovered individuals could, on average, be reinfected approximately every 18 months. Since omicron, however, case reports of reinfections just months, and sometimes weeks, after recovery have become common.  If you're looking to model a variety of plausible scenarios for a future without NPIs, you might want to look at mortality estimates over a given time frame with ```infections_per_year``` set to values between 0.25 and 2.

### 1 - CODE
Highlight the cell below and type shift + enter

In [547]:
def calc_mortality(age, years, infections_per_year, init_ifr=0, adjust_ifr_for_age=1.11, protected_risk=.61):
    if init_ifr == 0:
        _0_17 = [(20/1_000_000)/18 * 1.26956**i for i in range(19)]
        _18_49 = [_0_17[-1] * 1.09437**i for i in range(1, 33)]
        _50_64 = [_18_49[-1] * 1.16299**i for i in range(1, 16)]
        _65_79 = [_50_64[-1] * 1.211304**i for i in range(1, 16)]
        ifrs = _0_17 + _18_49 + _50_64 + _65_79
        ifr_dict = {i: ifrs[i] for i in range(len(ifrs))}
    l = [ifr_dict[age], ifr_dict[age]*protected_risk]
    for i in range(1, years+1):
        l.append(1-(1-(ifr_dict[age+i]*(protected_risk)))**infections_per_year)
    
    return sum(l), ifr_dict

### 2 - PARAMETERS 
Enter your ```age``` and a value for ```years``` and ```infections_per_year``` in the cell below, then 
type shift + enter

In [554]:
age = 45
years = 10
infections_per_year = 2/3

### 3 - CALL
Highlight the cell below and type shift + enter

In [555]:
print(f'A conservative estimate of cumulative mortality risk for a {age} year old over the next {years} year(s), if they are infected {infections_per_year:.2f} time(s) per year, is {calc_mortality(age, years, infections_per_year)[0]*100:.1f}%')
print(f'In other words, your cumulative risk of mortality over this period is {round(calc_mortality(age, years, infections_per_year)[0]/calc_mortality(age, years, infections_per_year)[1][age], 1)} times greater than your risk from a single infection.')

A conservative estimate of cumulative mortality risk for a 45 year old over the next 10 year(s), if they are infected 0.67 time(s) per year, is 0.9%
In other words, your cumulative risk of mortality over this period is 9.4 times greater than your risk from a single infection.


### EXPLANATION

This calculator works like this:  if you are relatively young and healthy, the likelihood that you'll end up dying from a covid infection is relatively small (on the order of 0.0005 for a healthy 38 year old), and the mortality risk after $x$ number of infections is   

$0.0005 + (1-((1-(0.0005*0.61))^{x-1}))$   

where 0.61 is the factor that represents the 39% reduction in mortality risk conferred from a previous infection.  So, for a relatively young, healthy person, the cumulative mortality risk of, for example, 6 infections, would be   

$0.0005 + (1-(1-(0.0005)*.61)^5) = 0.002$

Now, since age is likely the most significant factor in determining the mortality risk for most people, and since the risk of mortality increases considerably as we age (from about 0.00008 at age 18 to about 0.09 for the 65+ age group), in order to estimate your risk of mortality over $x$ number of years, we have to account for the fact that your mortality risk will gradually increase over those $x$ years.  So, for the example above, if those six infections happened twice per year over a three year period, the calculation would look like this:  

First year mortality risk: $0.0005 + (0.0005*0.61) = 0.0008$    
Second year mortality risk: $1-(1-(0.00055)*.61)^2 = 0.00067$  
Third year mortality risk: $1-(1-(0.00062)*.61)^2 = 0.00076$  

Notice that the IFR, which was 0.0005 during the first year, gradually increased to 0.00062 over the next two years. Also, notice that the first year was the riskiest thus far. This reflects the fact that, historically, recovery from an initial infection has imparted some protection against the most severe outcomes from subsequent infections.  Finally, notice how the risk begins creeping back up in year three as a result of the gradually increasing IFR.

To explain how to calculate the overall three-year mortality risk for this example, it might help to use a more familiar example.  If you roll 3 dice, what are the chances that none of the three dice is a 6?  We know that the odds of rolling a six are 1/6 for each die rolled, but what about the odds of getting at least one 6 when you roll three dice? It turns out that the odds of getting at least one 6 when you roll three dice is equal to the sum of the following probabilities:   
  
One six: $\binom{3}{1} \frac{1}{6}*\frac{5}{6}^2$  
Two sixes: $\binom{3}{2} \frac{1}{6}^2*\frac{5}{6}$  
Three sixes: $\binom{3}{3} \frac{1}{6}^3$  
  
Which is equal to $\sum_{k=1}^3 \binom{3}{k} \frac{1}{6}^{k}*\frac{5}{6}^{3-k}$ or one minus the probability of rolling zero sixes: $1-(\frac{5}{6}*\frac{5}{6}*\frac{5}{6})$. So, to calculate the cumulative mortality rate over three years, we multiply (1-0.0008), (1-0.00067) and (1-0.00076), and subtract the total from one      

$1-((1-0.0008)*(1-0.00067)*(1-0.00076)) = 0.002228$
  
and get a cumulative, IFR-adjusted mortality risk for 6 infections over three years of 0.00223, or about 0.223%.

# 2. PASC Risks

It was estimated that a quarter of the US population was infected with covid during the omicron wave. If we can expect, as a baseline scenario, similar rates of infection going forward, and similar biannual surges, then approximately 90% of the population will likely be infected **at least** once every four years. If these infections are normally distributed, however, then, more than 50% of the population can expect to be reinfected once every 18 months, and many people will be reinfected two or more times per year. Considering the withdrawal of NPIs and the normalization of mass infection, however, it is quite possible that these figures represent an underestimate. 

A recent meta-analysis published [here](https://www.sciencedirect.com/science/article/pii/S0889159121006516) estimates the prevalance of post-acute covid cognitive impairment at 22%, and chronic fatigue at 32%. This [Nature](https://www.nature.com/articles/s41586-022-04569-5_reference.pdf) article observes that recovered covid patients lose between 0.2 and 2% brain volume after an initial infection.  I've seen a number of other studies that show similar or corroborating findings.

If the physiological damage caused by infection and reinfection is cumulative, which we should assume it may be, then, in 10 years time, an average person, if they survive 7 infections, might lose between 1.4 and 13.2% of their initial brain volume (if that's even possible). Similarly, assuming, conservatively, that between 90 and 95% of long-covid sufferers have their symptoms resolve within 12 months, we can expect that, after ten years, a baseline of between 8.4 and 16.2% of the surviving population will suffer permanent or quasi-permanent impairment.  If, however, the prevalence of PASC increases over time as a function of damage sustained during previous infections, then these figures may significantly underestimate the long-term health risks of allowing covid to circulate.  

I don't think that we are prepared to deal with the financial and social repercussions of this level of chronic illness and disability. We're already reeling from the sudden withdrawal of a much smaller contingent from the workforce. So I personally fear that, at some point over the next decade, we'll collectively decide that we cannot afford to care for those that are too sick to work.

Anyway... to calculate the risk that you will eventually experience 'long-covid', enter a value for ```lc_prevalence``` in the cell marked **```2 - PARAMETERS```** below. As mentioned above, it seems that a conservative estimate might be somewhere between 20 and 30% (but I've seen credible estimates as high as 40%). Then enter a value for ```pct_recover``` (a conservative estimate might be 0.95, meaning that 95% of all people suffering PASC fully recover within 12 months).

### 1 - CODE
Highlight the cell below and type shift + enter

In [577]:
def estimate_pasc(lc_prevalence, years):
    risk = 1-(1-lc_prevalence)**mth.ceil(years*infections_per_year)
    p_risk = 1-(1-(lc_prevalence*(1-pct_recover)))**mth.ceil(years*infections_per_year)
    print(f'If you survive ~ {infections_per_year:.2f} infection(s) per year over the next {years} year(s), the risk that you will experience prolonged post-acute sequalae at some point is about {risk*100:.1f}%')
    print(f'Very conservatively, the chances that you will suffer some long-term or permanent functional impairment are about {p_risk*100:.1f}%')

### 2 - PARAMETERS 
Enter values for ```lc_prevalence``` in the cell below, then type shift + enter

In [578]:
lc_prevalence = 0.25
pct_recover = 0.95

### 3 - CALL
Highlight the cell below and type shift + enter

In [579]:
estimate_pasc(lc_prevalence, years)

If you survive ~ 0.67 infection(s) per year over the next 10 year(s), the risk that you will experience prolonged post-acute sequalae at some point is about 86.7%
Very conservatively, the chances that you will suffer some long-term or permanent functional impairment are about 8.4%
